In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [3]:
input_shape = (200, 200, 3)

In [4]:
model = Sequential()


# Add convolutional layers with pooling and dropout
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

# Add a fully connected layer with dropout
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))

# Add the output layer with sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 99, 99, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 99, 99, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 48, 48, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 48, 48, 64)        0

In [6]:
#cancel model
#too much to train

In [1]:
import numpy as np
import os
import cv2
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split

In [2]:
data_path = 'data_svm/'
img_size = (200, 200)
X = []
for filename in os.listdir(data_path):
    img = cv2.imread(os.path.join(data_path,filename))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, img_size)
    X.append(img)
X = np.array(X)



In [3]:
X_train, X_val = train_test_split(X, test_size=0.1)

# flatten
X_train = X_train.reshape(X_train.shape[0], -1)
X_val = X_val.reshape(X_val.shape[0], -1)


In [4]:
model = OneClassSVM(kernel='rbf', nu=0.1)
model.fit(X_train)

OneClassSVM(nu=0.1)

In [5]:
import pickle
with open('leaf_or_not' , 'wb') as f:
    pickle.dump(model,f)

In [6]:
y_val_pred = model.predict(X_val)

In [7]:
acc = (y_val_pred == 1).sum() / y_val_pred.shape[0]
print("Accuracy on validation set:", acc)

Accuracy on validation set: 0.8943070730304773


In [13]:
#cancel coz 1.1gb pkl file size..too slow to process------------------------------------------------------------------


In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
input_shape = (200, 200, 3)

In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow import keras
from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPool2D,AveragePooling2D,Dense,Flatten,ZeroPadding2D,BatchNormalization,Activation,Add,Input,Dropout,GlobalAveragePooling2D
from keras.initializers import glorot_uniform
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import regularizers
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
base_model_tf=ResNet50(include_top=False,weights='imagenet',input_shape=(200,200,3),classes=62)
base_model_tf.trainable=False
pt=Input(shape=(200,200,3))
func=tf.cast(pt,tf.float32)
x=preprocess_input(func) 
model_resnet=base_model_tf(x,training=False)
model_resnet=GlobalAveragePooling2D()(model_resnet)

model_resnet=Dense(32,activation='relu')(model_resnet)
model_resnet=Dense(1,activation='sigmoid')(model_resnet)
model=Model(inputs=pt,outputs=model_resnet)
model.summary()
model: "model"

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 tf.cast (TFOpLambda)        (None, 200, 200, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 200, 200, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 200, 200, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                      

In [5]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])


In [11]:
datagen = ImageDataGenerator(validation_split=0.1)
batch_size = 32
train_generator = datagen.flow_from_directory(
        'data_leaf_or_not',
        target_size=(224,224),
        batch_size=32,
        class_mode='binary',
        color_mode = 'rgb',
        subset='training',
        )
validation_generator = datagen.flow_from_directory(
        'data_leaf_or_not',
        target_size=(224,224),
        batch_size=32,
        class_mode='binary',
        color_mode='rgb', 
        subset='validation')

Found 39126 images belonging to 1 classes.
Found 4347 images belonging to 1 classes.


In [11]:
callback = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=3)
history = model.fit(train_generator, epochs=1, validation_data=validation_generator, callbacks=[callback])

612/612 [==============================] - 900s 1s/step - loss: 6.8481e-10 - accuracy: 1.0000 - val_loss: 7.3716e-10 - val_accuracy: 1.0000


In [14]:
# model.save('leaf_or_not', save_format = 'h5')

In [15]:
import cv2 
import numpy as np


In [26]:
im = cv2.imread('D:/gfg_hack/dataset_Augmented/test/PotatoHealthy2.JPG')
im = cv2.resize(im, (200,200))
im_tensor = np.expand_dims(im, axis=0)
im_tensor.shape

(1, 200, 200, 3)

In [27]:
pred = model.predict(im_tensor)

1/1 [==============================] - 0s 19ms/step


In [28]:
pred
print(pred)
print(0 if pred<0.7 else 1)

[[4.8745984e-15]]
0


In [29]:
#overfit

In [14]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
for layer in base_model.layers:
    layer.trainable = False
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.models.Model(inputs=base_model.input, outputs=output)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

 block_3_expand_relu (ReLU)     (None, 56, 56, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

In [15]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:
callback = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=5)
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // batch_size,
                    epochs=10,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples // batch_size,
                   callbacks=[callback])

Epoch 1/10
 369/1222 [========>.....................] - ETA: 11:53 - loss: 0.0167 - accuracy: 1.0000

In [115]:
import cv2 
import numpy as np
import pickle
model1 = pickle.load(open('leaf_or_not_svm', 'rb'))
im = cv2.imread('D:/gfg_hack/dataset_Augmented/test/gojo_billa_bg.png')
im = cv2.resize(im, (200,200))
im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im_tensor = np.reshape(im, (1,-1))

pred = model1.predict(im_tensor)
pred
print(pred)
print(0 if pred<0.7 else 1)

[-1]
0


In [28]:
import numpy as np
import os
import cv2
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split

In [46]:
data_path = 'data_leaf_or_not/'
img_size = (100, 100)
X = []
for filename in os.listdir(data_path):
    img = cv2.imread(os.path.join(data_path,filename))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, img_size)
    X.append(img)
X = np.array(X)


In [47]:
X_train, X_val = train_test_split(X, test_size=0.1)

# flatten
X_train = X_train.reshape(X_train.shape[0], -1)
X_val = X_val.reshape(X_val.shape[0], -1)


In [48]:
model_svm = OneClassSVM(kernel='rbf', nu=0.1)
model_svm.fit(X_train)

OneClassSVM(nu=0.1)

In [49]:
import pickle
with open('leaf_or_not' , 'wb') as f:
    pickle.dump(model_svm,f)

In [50]:
y_val_pred = model_svm.predict(X_val)

In [51]:
acc = (y_val_pred == 1).sum() / y_val_pred.shape[0]
print("Accuracy on validation set:", acc)

Accuracy on validation set: 0.8925399644760214


In [60]:
import cv2 
import numpy as np
import pickle
model1 = pickle.load(open('leaf_or_not', 'rb'))
im = cv2.imread('D:/gfg_hack/dataset_Augmented/test/7.jpg')
im = cv2.resize(im, (100,100))
im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im_tensor = np.reshape(im, (1,-1))

pred1 = model1.predict(im_tensor)
pred1
print(pred1)
print(0 if pred1<0.7 else 1)

[-1]
0
